In [1]:
from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten,Conv1D,MaxPooling1D
from keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer


Using TensorFlow backend.


In [2]:
df = pd.read_csv("dataset.csv",encoding='latin-1')


In [3]:
df.head()

,SentimentText,Sentiment
0,"first think another Disney movie, might good, ...",1
1,"Put aside Dr. House repeat missed, Desperate H...",0
2,"big fan Stephen King's work, film made even gr...",1
3,watched horrid thing TV. Needless say one movi...,0
4,truly enjoyed film. acting terrific plot. Jeff...,1


In [4]:
x_train=df.iloc[:,0].values

In [5]:
y_train=df.iloc[:,1].values

In [6]:
from keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords
for i in range (len(x_train)):
    x_train[i]=re.sub(r'\W',' ',x_train[i])
    x_train[i]=re.sub(r'\s+',' ',x_train[i])


In [7]:
max_words = 5000 # We will only consider the 10K most used words in this dataset

tokenizer = Tokenizer(num_words=max_words) # Setup
tokenizer.fit_on_texts(x_train) # Generate tokens by counting frequency
sequences = tokenizer.texts_to_sequences(x_train) # Turn text into sequence of numbers
word_index = tokenizer.word_index


    


In [8]:
x_train=pad_sequences(sequences,maxlen=200,padding='post',truncating='post',value=0)

In [9]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=0)


In [10]:
from keras.layers import GlobalMaxPooling1D,Embedding,SpatialDropout1D
model=Sequential()
model.add(Embedding(5000,128,input_length=200))
model.add(SpatialDropout1D(0.2))

model.add(Conv1D(64,3,activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

In [11]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=3,batch_size=64,validation_data=(x_val, y_val),validation_split=0.2)
model.summary()

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 35s 2ms/step - loss: 0.5206 - acc: 0.7391 - val_loss: 0.3498 - val_acc: 0.8494
Epoch 2/3
20000/20000 [==============================] - 37s 2ms/step - loss: 0.3353 - acc: 0.8565 - val_loss: 0.3022 - val_acc: 0.8752
Epoch 3/3
20000/20000 [==============================] - 33s 2ms/step - loss: 0.2573 - acc: 0.8970 - val_loss: 0.2974 - val_acc: 0.8716
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 128)          640000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 200, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 198, 64)           24640     
_________________________________________________________________
global_max_pool

In [39]:
y_text = 'Medha is the best and she talks sense is interesting.'
seq = tokenizer.texts_to_sequences([y_text])
seq = pad_sequences(seq, maxlen=200)
prediction = model.predict(seq)
print('positivity:',prediction)

('positivity:', array([[0.64657104]], dtype=float32))
